<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Analysis-Migration-Discourse-as-Social-Networks" data-toc-modified-id="Analysis-Migration-Discourse-as-Social-Networks-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Analysis Migration Discourse as Social Networks</a></span></li><li><span><a href="#Marijn's-analysis" data-toc-modified-id="Marijn's-analysis-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Marijn's analysis</a></span></li><li><span><a href="#My-analysis" data-toc-modified-id="My-analysis-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>My analysis</a></span></li></ul></div>

## Analysis Migration Discourse as Social Networks

Method:
- generate networks per period, each person or organisation is a node
    - **Shape**: different types of nodes have different shapes: organisation, academic, technocrat
    - **Color**: use different colours to represent different nationalities, grey for unknown nationalities, black for organisations
    

Vragen:

- Beijer, G. Beijer en G.O.K. Beijer zijn dezelfde persoon?
- sommige contributors van de studies staan niet in de personenlijst. Hoe categoriseren we die?



In [ ]:
import sys
#!python -m pip install pandas networkx
%pip install matplotlib seaborn

In [3]:
sys.path.append('/data/home/jupyter-jdh-artikel/.local/lib/python3.7/site-packages')
%reload_ext autoreload
%autoreload 2

In [4]:
import pandas as pd
import networkx as nx
import json


## Preparation of data

In [5]:
from scripts.network_analysis import retrieve_spreadsheet_records

def lowercase_headers(records:list):
    return [dict((k.lower(), v) for k,v in record.items()) for record in records] 

entity_records = retrieve_spreadsheet_records(record_type='entities')
entity_records = lowercase_headers(entity_records)
#print('Number of records:' , len(entity_records))


In [6]:
import json

for record in entity_records:
    print(json.dumps(record, indent=4))

{
    "organisation": "REMP",
    "period": "1952",
    "prs_id": "1",
    "prs_surname": "Beijer",
    "prs_infix": "",
    "prs_initials": "G.",
    "prs_function": "demographer, The Hague",
    "prs_category": "academic",
    "prs_country": "NL",
    "prs_role1": "founder",
    "prs_role2": "member_MC",
    "prs_role3": "secretary-editor"
}
{
    "organisation": "REMP",
    "period": "1952",
    "prs_id": "2",
    "prs_surname": "Groenman",
    "prs_infix": "",
    "prs_initials": "Sj.",
    "prs_function": "sociologist, Leiden",
    "prs_category": "academic",
    "prs_country": "NL",
    "prs_role1": "founder",
    "prs_role2": "member_MC",
    "prs_role3": "vice-chair_BoD"
}
{
    "organisation": "REMP",
    "period": "1952",
    "prs_id": "3",
    "prs_surname": "Zeegers",
    "prs_infix": "",
    "prs_initials": "G.H.L.",
    "prs_function": "sociologist, Nijmegen",
    "prs_category": "academic",
    "prs_country": "NL",
    "prs_role1": "founder",
    "prs_role2": "member_MC"

In [7]:
# index each person entity with their category (academic, technocrat)
def get_entity_name(entity: dict, ):
    name_labels = ['prs_surname', 'prs_infix', 'prs_initials']
    if entity['prs_infix'] != '':
        return f"{entity['prs_surname']}, {entity['prs_infix']}, {entity['prs_initials']}"
    else:
        return f"{entity['prs_surname']}, {entity['prs_initials']}"

entity_roles = {get_entity_name(record): [record['prs_role1'], record['prs_role2'], record['prs_role3']] for record in entity_records}
for k in entity_roles:
    nr = [e for e in entity_roles[k] if e !='']
    entity_roles[k]=nr
#entity_roles
entity_category = {get_entity_name(entity = record): record.get('prs_category') or 'unknown' for record in entity_records}

In [8]:
from scripts.network_analysis import retrieve_spreadsheet_records

relationship_records = retrieve_spreadsheet_records(record_type='relationships')
#len(relationship_records)

In [9]:
pd.DataFrame().from_records(relationship_records).columns

Index(['series', 'volume', 'year', 'article_author1_surname',
       'article_author1_infix', 'article_author1_initials',
       'article_author2_surname', 'article_author2_infix',
       'article_author2_initials', 'preface_author1_surname',
       'preface_author1_infix', 'preface_author1_initials',
       'preface_author2_surname', 'preface_author2_infix',
       'preface_author2_initials', 'intro_author1_surname',
       'intro_author1_infix', 'intro_author1_initials',
       'intro_author2_surname', 'intro_author2_infix',
       'intro_author2_initials', 'executor_org', 'funder', 'client',
       'editor_surname', 'editor_infix', 'editor_initials', 'volume_title'],
      dtype='object')

In [10]:
from itertools import chain


In [11]:
er = list(set(chain.from_iterable(entity_roles.values())))
er

['AB',
 'Dep_dir_ICEM',
 'Correspondent',
 'editor',
 'vice-chair_BoD',
 'founder',
 'member_MC',
 'member_BoD',
 'member _MC',
 'chair_MC',
 'chair_BoD']

In [12]:
{'vice-chair_BoD':'REMP director',
 'Dep_dir_ICEM':'ICEM',
 'chair_MC':'REMP board',
 'AB':"REMP board",
 'Correspondent':"correspondent",
 'member _MC':"REMP board",
 'chair_BoD':"REMP director",
 'member_BoD':"REMP director",
 'founder':"founder",
 'editor':"editor",
 'member_MC':"REMP board"}

roled = {'author':['article_author1_surname','article_author2_surname'],
 'preface_author':['preface_author1_surname','preface_author2_surname'],
 'intro_author':['intro_author1_surname','intro_author2_surname'],
 'executor':['executor_org'], 
 'funder':['funder'],
 'client':['client'],
 'editor':['editor_surname'],
 'unknown':['']}

rroled = {w:k for k,v in roled.items() for w in v}
rroled
# roles = ['executor', 'funder', 'board', 'client',
#        'editor', 'author', 'preface_writer', 'intro_writer']

{'article_author1_surname': 'author',
 'article_author2_surname': 'author',
 'preface_author1_surname': 'preface_author',
 'preface_author2_surname': 'preface_author',
 'intro_author1_surname': 'intro_author',
 'intro_author2_surname': 'intro_author',
 'executor_org': 'executor',
 'funder': 'funder',
 'client': 'client',
 'editor_surname': 'editor',
 '': 'unknown'}

In [13]:
categorized_persons = retrieve_spreadsheet_records("categories")
categorized_persons = lowercase_headers(categorized_persons)

In [14]:
from collections import defaultdict, Counter

from scripts.network_analysis import extract_record_entities

def get_entity_category(entity: dict):
    if entity.get('entity_name') in entity_category:
        return entity_category[entity['entity_name']] 
    else:
        return 'unknown'


In [15]:
from collections import defaultdict, Counter
record_entities = defaultdict(list)
entity_count = Counter()
entity_role_count = Counter()
for ri, record in enumerate(relationship_records):
    entities = extract_record_entities(record)
    record_entities[ri].append(entities)
    entity_count.update([entity['entity_name'] for entity in entities if 'entity_name' in entity])
    entity_role_count.update([entity['entity_role'] + ' ' + entity['entity_name'] for entity in entities if 'entity_name' in entity])
    for entity in entities:
        if entity['entity_type'] == 'person':
            entity['entity_type'] = get_entity_category(entity)
        print(entity)
    print(ri)
    
    

{'entity_name': 'RCE', 'entity_role': 'executor_org', 'entity_type': 'organisation'}
{'entity_name': 'ICEM', 'entity_role': 'funder', 'entity_type': 'organisation'}
{'entity_name': 'RCE', 'entity_role': 'client', 'entity_type': 'organisation'}
{'entity_name': 'Hofstede, B.P.', 'entity_role': 'article_author', 'entity_type': 'unknown'}
{'entity_name': 'Groenman, Sj.', 'entity_role': 'preface_author', 'entity_type': 'academic'}
{'entity_type': 'publication', 'series': 'Studies over Nederlandse emigratie', 'volume': '1', 'year': 1958, 'volume_title': 'De gaande man : gronden van de emigratiebeslissing : rapport naar aanleiding van een onderzoek / verricht in opdracht van de Regeringscommissaris voor de Emigratie '}
0
{'entity_name': 'REMP', 'entity_role': 'executor_org', 'entity_type': 'organisation'}
{'entity_name': 'ICEM', 'entity_role': 'funder', 'entity_type': 'organisation'}
{'entity_name': 'RCE', 'entity_role': 'client', 'entity_type': 'organisation'}
{'entity_name': 'Frijda, N.H.',

In [16]:
record_entities

defaultdict(list,
            {0: [[{'entity_name': 'RCE',
                'entity_role': 'executor_org',
                'entity_type': 'organisation'},
               {'entity_name': 'ICEM',
                'entity_role': 'funder',
                'entity_type': 'organisation'},
               {'entity_name': 'RCE',
                'entity_role': 'client',
                'entity_type': 'organisation'},
               {'entity_name': 'Hofstede, B.P.',
                'entity_role': 'article_author',
                'entity_type': 'unknown'},
               {'entity_name': 'Groenman, Sj.',
                'entity_role': 'preface_author',
                'entity_type': 'academic'},
               {'entity_type': 'publication',
                'series': 'Studies over Nederlandse emigratie',
                'volume': '1',
                'year': 1958,
                'volume_title': 'De gaande man : gronden van de emigratiebeslissing : rapport naar aanleiding van een onderzoek / verricht

In [17]:
def make_nodes(entity):
    return [n.get('entity_name') for n in entity if n.get('entity_name')]
    
def make_link_from_entity(entity, revnodelist):
    counter = []
    authors = [n.get('entity_name') for n in entity if n.get('entity_role')=='article_author']
    links = []
    for aut in authors:
        autnr = revnodelist[aut]
        counter.append(autnr)
        for node in entity:
            if node.get('entity_role') != 'article_author':
                if node.get('entity_role'): # we don't include titles
                    category = node.get('entity_role') or "unknown"
                    target = revnodelist[node.get('entity_name')]
                    graphnode = rempgraph.nodes()[target]
                    if graphnode.get('category'):
                        graphnode['category'].append(category)
                    else:
                        graphnode['category']= [category]
                    link = (autnr, target, {"link_type": node.get('entity_role') or 'unknown'})
                    links.append(link)
                    counter.append(target)
    
    return links, counter

In [18]:
nodelist = []
for rentity in record_entities:
    nodelist.extend(make_nodes(record_entities[rentity][0]))
nodelist = list(set(nodelist))
revnodelist = {}
rempgraph = nx.Graph()
for node in enumerate(nodelist):
    rempgraph.add_node(node[0], id=node[0], name=node[1])
    revnodelist[node[1]]=node[0]

In [19]:
rempgraph.nodes

NodeView((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134))

In [20]:
linklist = []
counter = Counter()
for rentity in record_entities:
    links, cntr = make_link_from_entity(record_entities[rentity][0], revnodelist)
    linklist.extend(links)
    counter.update(cntr)

In [21]:
counter.most_common(10)

[(46, 160),
 (108, 29),
 (71, 5),
 (94, 5),
 (115, 5),
 (38, 4),
 (68, 4),
 (122, 4),
 (23, 4),
 (25, 3)]

In [22]:
for node in rempgraph.nodes():
    n = rempgraph.nodes()[node]
    value = counter[n['id']]
    n['value'] = value
    

In [23]:
import networkx as nx
rempgraph.add_edges_from(linklist)
nx.draw(rempgraph, pos=nx.spring_layout(rempgraph, weight=value))

## Marijn's analysis

In [24]:
for entity, count in entity_count.most_common():
    print(f'{entity: <30}{count: >3}')

Beijer, G.                    147
REMP                           25
RCE                             5
Groenman, Sj.                   5
Hofstee, E.W.                   5
ICEM                            4
Edding, F.                      4
Sauvy, A.                       3
Zeegers, G.H.L.                 3
Isaac, J.                       3
Hack, H.                        3
Mol, J.J.                       3
Radspieler, T.                  3
Zubrzycki, J.                   3
Richardson, A.                  3
Hofstede, B.P.                  2
Frijda, N.H.                    2
Haveman, B.W.                   2
Wentholt, R.                    2
Gadolin, de, A.                 2
Petersen, W.                    2
Mill, van, A.N.                 2
Bastos de Avila, F.             2
Ipsen, G.                       2
Lehmkuhler, M.                  2
Nixon, J.W.                     2
Mast, van der, W.               2
Beld, van den, C.A.             2
Appleyard, R.T.                 2
Timlin, M.F.  

In [25]:
for entity, count in entity_role_count.most_common():
    print(f'{entity: <30}{count: >3}')

editor Beijer, G.             135
executor_org REMP              24
article_author Beijer, G.      12
funder ICEM                     4
client RCE                      4
article_author Edding, F.       4
article_author Hofstee, E.W.    4
article_author Hack, H.         3
article_author Mol, J.J.        3
article_author Radspieler, T.   3
article_author Zubrzycki, J.    3
article_author Richardson, A.   3
article_author Hofstede, B.P.   2
preface_author Groenman, Sj.    2
article_author Frijda, N.H.     2
preface_author Haveman, B.W.    2
article_author Wentholt, R.     2
preface_author Sauvy, A.        2
article_author Gadolin, de, A.  2
article_author Petersen, W.     2
article_author Bastos de Avila, F.  2
article_author Lehmkuhler, M.   2
article_author Nixon, J.W.      2
article_author Mast, van der, W.  2
article_author Groenman, Sj.    2
article_author Isaac, J.        2
article_author Beld, van den, C.A.  2
article_author Appleyard, R.T.  2
article_author Timlin, M.F.     2
arti

In [26]:
from scripts.network_analysis import make_bibliographic_records

bib_records = make_bibliographic_records(relationship_records)
print(bib_records[0])
len(bib_records)


{'article_title': 'De gaande man : gronden van de emigratiebeslissing : rapport naar aanleiding van een onderzoek / verricht in opdracht van de Regeringscommissaris voor de Emigratie ', 'article_doi': None, 'article_author': 'Hofstede, B.P.', 'article_author_index_name': 'Hofstede, B.P.', 'article_author_affiliation': '', 'article_page_range': None, 'article_pub_date': '1958', 'article_pub_year': 1958, 'issue_section': 'article', 'issue_number': None, 'issue_title': None, 'issue_page_range': None, 'issue_pub_date': '1958', 'issue_pub_year': 1958, 'volume': '1', 'journal': 'Studies over Nederlandse emigratie', 'publisher': 'REMP'}


169

In [27]:
headers = [
    'article_title', 'article_doi', 
    'article_author', 'article_author_index_name', 'article_author_affiliation', 
    'article_page_range', 'article_pub_date', 'article_pub_year', 
    'issue_section', 'journal', 'volume', 'issue', 
    'issue_title', 'issue_page_range', 'issue_pub_date', 'issue_pub_year', 
    'publisher'
]

remp_records_file = '../data/remp-records.json'
with open(remp_records_file, 'wt') as fh:
    json.dump(bib_records, fh)

In [28]:
from scripts.network_analysis import generate_graph, add_entities, add_record_links

graph = generate_graph()

periods = [
    {'start': 1950, 'end': 1959}, 
    {'start': 1960, 'end': 1969},
    {'start': 1970, 'end': 1979},
    {'start': 1980, 'end': 1989},
]

period = periods[0]
for record in relationship_records:
    record['year'] = int(record['year'])
for ri, record in enumerate(sorted(relationship_records, key = lambda x: x['year'])):
    record['year'] = int(record['year'])
    if record['year'] < period['start'] or record['year'] > period['end']:
        continue
    print(record['year'])
    entities = extract_record_entities(record)
    for entity in entities:
        if entity['entity_type'] == 'person':
            entity['entity_type'] = get_entity_category(entity)
    named_entities = [entity for entity in entities if 'entity_name' in entity]
    print([entity['entity_name'] for entity in named_entities])
    add_entities(graph, named_entities)
    add_record_links(graph, named_entities)
    if ri == 100:
        break

graph.number_of_nodes()

1951
['REMP', 'Wander, H.', 'Bouman, P.J.', 'Beijer, G.']
adding link REMP Wander, H.
adding link REMP Bouman, P.J.
adding link REMP Beijer, G.
adding link Wander, H. Bouman, P.J.
adding link Wander, H. Beijer, G.
adding link Bouman, P.J. Beijer, G.
1951
['REMP', 'Citroen, H.A.', 'Groenman, Sj.', 'Rappard, W.E.', 'Beijer, G.']
adding link REMP Citroen, H.A.
adding link REMP Groenman, Sj.
adding link REMP Rappard, W.E.
adding link REMP Beijer, G.
adding link Citroen, H.A. Groenman, Sj.
adding link Citroen, H.A. Rappard, W.E.
adding link Citroen, H.A. Beijer, G.
adding link Groenman, Sj. Rappard, W.E.
adding link Groenman, Sj. Beijer, G.
adding link Rappard, W.E. Beijer, G.
1951
['REMP', 'Edding, F.', 'Salin, E.', 'Beijer, G.']
adding link REMP Edding, F.
adding link REMP Salin, E.
adding link REMP Beijer, G.
adding link Edding, F. Salin, E.
adding link Edding, F. Beijer, G.
adding link Salin, E. Beijer, G.
1952
['REMP', 'Beijer, G.', 'Oudegeest, J.J.', 'Sauvy, A.', 'Beijer, G.']
adding 

94

In [29]:
graph.nodes()

NodeView(('REMP', 'Wander, H.', 'Bouman, P.J.', 'Beijer, G.', 'Citroen, H.A.', 'Groenman, Sj.', 'Rappard, W.E.', 'Edding, F.', 'Salin, E.', 'Oudegeest, J.J.', 'Sauvy, A.', 'Brink, van den , T.', 'Gadolin, de, A.', 'Zeegers, G.H.L.', 'HWvanLoon Fellowship, NL regering', 'Institute of International Education', 'Petersen, W.', 'Hofstee, E.W.', 'Davis, K.', 'Glass, D.V.', 'Mackenroth, G.', 'Nixon, J.W.', 'Lannes, X.', 'Mill, van, A.N.', 'Mast, van der, W.', 'Glinstra Bleeker, van, R.J.P.', 'Sjollema, B.Ch.', 'MacKay, M.C.J.', 'Dorpema, H.', 'Bentwich, N.', 'Beveridge, W.H.', 'Bastos de Avila, F.', 'Camillo de Oliveira, A.', 'Isaac, J.', 'Peltier, R.', 'Brink, van den, T', 'Meyers, H.G.', 'Godefroy, J.', 'Radspieler, T.', 'Weinberg, A.A.W.', 'Schmitz, J.P.G.', 'Beld, van den, C.A.', 'Hyrenius, H.', 'Mannoni, E.', 'Belshaw, H.', 'Olsson, B.', 'Geyl, W.F.', 'Steigenga, W.', 'Ipsen, G.', 'Brepohl, W.', 'Lehmkuhler, M.', 'Borrie, W.D.', 'Lewis, J.N.', 'Sauvon, E.A.', 'Maris, A.', 'Pressat, R.',

In [30]:
len(graph.edges())

170

In [31]:
graph.edges()

EdgeView([('REMP', 'Wander, H.'), ('REMP', 'Bouman, P.J.'), ('REMP', 'Beijer, G.'), ('REMP', 'Citroen, H.A.'), ('REMP', 'Groenman, Sj.'), ('REMP', 'Rappard, W.E.'), ('REMP', 'Edding, F.'), ('REMP', 'Salin, E.'), ('REMP', 'Oudegeest, J.J.'), ('REMP', 'Sauvy, A.'), ('REMP', 'Brink, van den , T.'), ('REMP', 'Gadolin, de, A.'), ('REMP', 'Zeegers, G.H.L.'), ('REMP', 'HWvanLoon Fellowship, NL regering'), ('REMP', 'Institute of International Education'), ('REMP', 'Petersen, W.'), ('REMP', 'Hofstee, E.W.'), ('REMP', 'Davis, K.'), ('REMP', 'Glass, D.V.'), ('REMP', 'Lannes, X.'), ('REMP', 'Mill, van, A.N.'), ('REMP', 'Bastos de Avila, F.'), ('REMP', 'Camillo de Oliveira, A.'), ('REMP', 'Isaac, J.'), ('REMP', 'Steigenga, W.'), ('REMP', 'Ipsen, G.'), ('REMP', 'Verwey-Jonker, H.'), ('REMP', 'Brackel, P.O.M.'), ('REMP', 'Klompe, M.'), ('REMP', 'Verkade, W.'), ('Wander, H.', 'Bouman, P.J.'), ('Wander, H.', 'Beijer, G.'), ('Bouman, P.J.', 'Beijer, G.'), ('Beijer, G.', 'Citroen, H.A.'), ('Beijer, G.', 

## My analysis

In [32]:
import matplotlib.pyplot as plt

import networkx as nx

In [33]:

import pandas as pd 

def categorize_node(n):
    cat = 0
    if n.get("is_academic")!= '':
        cat += 1
    if n.get("is_public_admistration") != '':
        cat += 2
    return cat


cp_df = pd.DataFrame().from_records(categorized_persons)
cp_df['norm_name'] = cp_df.apply(lambda x: get_entity_name(x), axis=1)
cp_df['orgcat'] = cp_df.apply(lambda x: categorize_node(x), axis=1)



In [34]:
cp_df.head()

,organisation,period_start,last_known_date,prs_id,prs_surname,prs_infix,prs_initials,prs_function,prs_category,is_academic,is_public_administration,sources,prs_country,prs_role1,prs_role2,prs_role3,remarks,norm_name,orgcat
0,REMP,1952,1983,1,Beijer,,G.,"demographer, The Hague",academic,yes,,,NL,founder,member_MC,secretary-editor,director-editor (1969),"Beijer, G.",3
1,REMP,1952,1969,2,Groenman,,Sj.,"sociologist, Leiden",academic,1947,1943-1950,https://nl.wikipedia.org/wiki/Sjoerd_Groenman ...,NL,founder,member_MC,vice-chair_BoD,,"Groenman, Sj.",3
2,REMP,1952,1969,3,Zeegers,,G.H.L.,"economist, sociologist, Nijmegen",academic,yes,1941-1950,https://www.ru.nl/kdc/bladeren/archieven-thema...,NL,founder,member_MC,member_BoD,,"Zeegers, G.H.L.",3
3,REMP,1952,1969,4,Hofstee,,E.W.,"sociologist, Wageningen",academic,yes,"yes, advisor 5 ministeries",http://resources.huygens.knaw.nl/bwn1880-2000/...,NL,founder,member_BoD,,,"Hofstee, E.W.",3
4,REMP,1952,1969,5,Bouman,,P.J.,"sociologist, Groningen",academic,yes,,"https://nl.wikipedia.org/wiki/P.J._Bouman, htt...",NL,member_BoD,,chair_BoD (1954),,"Bouman, P.J.",3


In [35]:

for n in rempgraph.nodes():
    r = cp_df.loc[cp_df.norm_name == rempgraph.nodes()[n]['name']]
    if len(r) > 0:
        rec = r[['organisation', 'period_start', 'last_known_date', 'norm_name', 'prs_country', 'orgcat']].to_dict('records')[0]
        rempgraph.nodes[n].update(rec)
        
#         print(rempgraph.nodes()[n]['name'], r)

In [36]:
rempgraph.nodes[1]

{'id': 1, 'name': 'Lannes, X.', 'value': 1}

In [37]:
for n in rempgraph.nodes():
    node = rempgraph.nodes()[n]
    if node.get('category'):
        node['category']= list(set(node['category']))
    else:
        node['category'] = ['unknown']
    node['category'] = ''.join([c[0] for c in node['category']])

In [38]:
from networkx.readwrite import json_graph
jsongraph = json_graph.node_link_data(rempgraph)
# json.dumps(json_graph)
with open('js/jsongraph.json', 'w') as outfl:
    json.dump(fp=outfl, obj=jsongraph)

In [39]:
json_graph2 = json_graph.node_link_data(rempgraph, attrs=dict(source='source', target='target',
                                                         name='id', nid='id', links='links'))
json_graph2

{'directed': False,
 'multigraph': False,
 'graph': {},
 'nodes': [{'id': 0, 'name': 'Clemens, R.', 'category': 'p', 'value': 1},
  {'id': 1, 'name': 'Lannes, X.', 'value': 1, 'category': 'u'},
  {'id': 2, 'name': 'Jorge, A.', 'value': 1, 'category': 'u'},
  {'id': 3,
   'name': 'Bouman, P.J.',
   'category': 'p',
   'value': 1,
   'organisation': 'REMP',
   'period_start': '1952',
   'last_known_date': '1969',
   'norm_name': 'Bouman, P.J.',
   'prs_country': 'NL',
   'orgcat': 3},
  {'id': 4, 'name': 'Balhana, A.Pilatti', 'value': 1, 'category': 'u'},
  {'id': 5, 'name': 'Richardson, A.', 'value': 3, 'category': 'u'},
  {'id': 6, 'name': 'Doczy, A.G.', 'value': 1, 'category': 'u'},
  {'id': 7, 'name': 'Lyon, A.V.', 'value': 1, 'category': 'u'},
  {'id': 8, 'name': 'Schoenberg, H.W.', 'value': 1, 'category': 'u'},
  {'id': 9, 'name': 'Rosenfield, H.N.', 'value': 2, 'category': 'u'},
  {'id': 10, 'name': 'Timlin, M.F.', 'value': 2, 'category': 'u'},
  {'id': 11, 'name': 'Camillo de Oli

In [40]:
rempgraph

In [41]:
nodes = graph.nodes
nodes['REMP']

{'name': 'REMP', 'entity_type': 'organisation'}

In [42]:
centralities = nx.eigenvector_centrality(graph)


In [43]:
import nx_altair as nxa
import altair as alt

In [44]:
nxa.draw_networkx_nodes( G=rempgraph,
    pos=pos,
    node_size="centrality",
    node_color='entity_type',
    edge_color='community',
    node_label=True,
    cmap='accent',
    node_tooltip=['name', 'centrality', "community"]
)

NameError: name 'pos' is not defined

In [45]:
communities = list(nx.community.label_propagation_communities(graph))
my_communities = {
 'ICEM':{'Appleyard, R.T.',
  'Balhana, A.Pilatti',
  'Beijer, G.',
  'Beld, van den, C.A.',
  'Belshaw, H.',
  'Borrie, W.D.',
  'Cidor, H.A.',
  'Dorpema, H.',
  'Fagley, R.',
  'Geyl, W.F.',
  'Glinstra Bleeker, van, R.J.P.',
  'Gottmann, J.',
  'Hack, H.',
  'Herberle, R.',
  'Heyn, H.',
  'Hyrenius, H.',
  'Kraak, J.H.',
  'Luckham, B.',
  'Lyon, A.V.',
  'MacKay, M.C.J.',
  'Mackenroth, G.',
  'Mannoni, E.',
  'Maris, A.',
  'Mast, van der, W.',
  'Mol, J.J.',
  'Nixon, J.W.',
  'Olsson, B.',
  'Orleans, L.A.',
  'Paulson, B.',
  'Peltier, R.',
  'Pressat, R.',
  'Price, C.A.',
  'Putten, v.d., A.',
  'Rees, E.',
  'Reid, I.D.A.',
  'Richardson, A.',
  'Rosenfield, H.N.',
  'Rubin, E.',
  'Schmitz, J.P.G.',
  'Sjollema, B.Ch.',
  'Starr, M.',
  'Timlin, M.F.',
  'Weinberg, A.A.W.',
  'Wentholt, R.',
  'Witcover, J.',
  'Witkamp, T.','ICEM'},
 'REMP':{'Beijer, G.',
         'Bastos de Avila, F.',
  'Bouman, P.J.',
  'Brackel, P.O.M.',
  'Brink, van den , T.',
  'Brink, van den, T',
  'Camillo de Oliveira, A.',
  'Citroen, H.A.',
  'Davis, K.',
  'Edding, F.',
  'Gadolin, de, A.',
  'Glass, D.V.',
  'Groenman, Sj.',
  'HWvanLoon Fellowship, NL regering',
  'Hofstee, E.W.',
  'Institute of International Education',
  'Ipsen, G.',
  'Isaac, J.',
  'Klompe, M.',
  'Lannes, X.',
  'Mill, van, A.N.',
  'Oudegeest, J.J.',
  'Petersen, W.',
  'REMP',
  'Rappard, W.E.',
  'Salin, E.',
  'Sauvy, A.',
  'Steigenga, W.',
  'Verkade, W.',
  'Verwey-Jonker, H.',
  'Wander, H.',
  'Zeegers, G.H.L.',
  'Hofstede, B.P.', 
  'RCE'},
'rest': {'Clemens, R.', 'Zubrzycki, J.', 'Godefroy, J.', 'Meyers, H.G.', 'Patijn, C.L.', 'Witkamp, F.Th.',
         'Brepohl, W.', 'Lehmkuhler, M.','Lewis, J.N.', 'Sauvon, E.A.','Neher, L.', 'Radspieler, T.',
         'Bentwich, N.', 'Beveridge, W.H.'}}

In [46]:
for f in graph.edges():
    for i, c in my_communities.items():
         if f[0] and f[1] in c:
            comty = graph.edges[f].get('community') or ''
            comty = ', '.join([comty,i])
            graph.edges[f].update({"community" : comty,
                                })

In [47]:
graph_centralities = nx.eigenvector_centrality(graph)

In [48]:
#first reset community on graph nodes
for f in graph.nodes():
    graph.nodes[f].update({"community" : ''})

for f in graph.nodes():
    for i, c in my_communities.items():
        if f in c:
            comty = graph.nodes[f].get('community') or ''
            comty = '/'.join([comty,i])[1:]
            label = ''
            if comty == "rest":
                label = f
            graph.nodes[f].update({"community" : comty,
                                "centrality" : graph_centralities[f]*2,
                                "name" : f,
                                "label" : label
                               })

In [49]:


pos = nx.spring_layout(graph)

            
chart = nxa.draw_networkx(
    G=graph,
    pos=pos,
    node_size="centrality",
    node_color='entity_type',
    edge_color='community',
    cmap='accent',
    node_tooltip=['name', 'centrality', "community"],
    node_label='label'
)

chart.configure_view(width=1000, height=800, )


alt.LayerChart(...)

In [51]:
import nx_altair as nxa
import altair as alt

# # Compute positions for viz.
# pos = nx.kamada_kawai_layout(graph)

# # Draw the graph using Altair
# viz = nxa.draw_networkx(graph, pos=pos)
# brush = alt.selection_interval(encodings=['x', 'y'])
# color = alt.Color('viable:N',  legend=None)




#pos = nx.spring_layout(graph)
partition = community_louvain.best_partition(graph)
pos = community_layout(graph, partition)


chart = nxa.draw_networkx(
    G=graph,
    pos=pos,
    node_size="centrality",
    node_color='color',
    edge_color='community',
    cmap='accent',
    node_tooltip=['name', 'entity_type']
)

chart.configure_view(width=800, height=400)

# Get the node layer
edges = chart.layer[0]
nodes = chart.layer[1]

# Build a brush
brush = alt.selection_interval(encodings=['x', 'y'])
color = alt.Color('community:N')

# Condition nodes based on brush
nodes = nodes.encode(
    fill=alt.condition(brush, color, alt.value('gray')),
).add_selection(
    brush
)

# Create a bar graph to show highlighted nodes.
bars = alt.Chart(nodes.data).mark_bar().encode(
    x=alt.X('count()', scale=alt.Scale(domain=(0,50))),
    y='community',
    color='entity_type',
).transform_filter(
    brush
)

alt.vconcat(edges+nodes, bars).configure_view(width=1000, height=600, )




NameError: name 'community_layout' is not defined

In [52]:
with open('js/testnxgraph.json', 'w') as tg:
    json.dump(fp=tg, obj=nx.json_graph.node_link_data(graph))

In [53]:
#nx.draw_spring(graph, with_labels=True, font_weight='bold')
pos = nx.spring_layout(graph, k=0.9)
#nx.draw_networkx_nodes(graph, pos)
#nx.draw_spectral(graph, with_labels=True, font_weight='bold')
pos

{'REMP': array([-0.13906946,  0.12238529]),
 'Wander, H.': array([-0.2264963 , -0.56423917]),
 'Bouman, P.J.': array([-0.39392549, -0.1600646 ]),
 'Beijer, G.': array([ 0.01865671, -0.03583434]),
 'Citroen, H.A.': array([-0.34002505,  0.12019085]),
 'Groenman, Sj.': array([0.04509742, 0.54678798]),
 'Rappard, W.E.': array([0.27378414, 0.12444892]),
 'Edding, F.': array([-0.63079603, -0.13508866]),
 'Salin, E.': array([-0.80034321,  0.40066473]),
 'Oudegeest, J.J.': array([-0.61196235,  0.67780581]),
 'Sauvy, A.': array([-0.2760751 ,  0.20977715]),
 'Brink, van den , T.': array([-0.03603965,  0.491438  ]),
 'Gadolin, de, A.': array([-0.05487905,  0.71117173]),
 'Zeegers, G.H.L.': array([-0.14270244, -0.10328325]),
 'HWvanLoon Fellowship, NL regering': array([-0.059593  ,  0.15901395]),
 'Institute of International Education': array([-0.3420835,  0.4541541]),
 'Petersen, W.': array([-0.2111547 ,  0.47546552]),
 'Hofstee, E.W.': array([-0.10750066,  0.02625536]),
 'Davis, K.': array([-0.4

In [54]:
import sys
!{sys.executable} -m pip install python-louvain

Defaulting to user installation because normal site-packages is not writeable


In [55]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

def community_layout(g, partition):
    """
    Compute the layout for a modular graph.


    Arguments:
    ----------
    g -- networkx.Graph or networkx.DiGraph instance
        graph to plot

    partition -- dict mapping int node -> int community
        graph partitions


    Returns:
    --------
    pos -- dict mapping int node -> (float x, float y)
        node positions

    """

    pos_communities = _position_communities(g, partition, scale=3.)

    pos_nodes = _position_nodes(g, partition, scale=1.)

    # combine positions
    pos = dict()
    for node in g.nodes():
        pos[node] = pos_communities[node] + pos_nodes[node]

    return pos

def _position_communities(g, partition, **kwargs):

    # create a weighted graph, in which each node corresponds to a community,
    # and each edge weight to the number of edges between communities
    between_community_edges = _find_between_community_edges(g, partition)

    communities = set(partition.values())
    hypergraph = nx.DiGraph()
    hypergraph.add_nodes_from(communities)
    for (ci, cj), edges in between_community_edges.items():
        hypergraph.add_edge(ci, cj, weight=len(edges))

    # find layout for communities
    pos_communities = nx.spring_layout(hypergraph, **kwargs)

    # set node positions to position of community
    pos = dict()
    for node, community in partition.items():
        pos[node] = pos_communities[community]

    return pos

def _find_between_community_edges(g, partition):

    edges = dict()

    for (ni, nj) in g.edges():
        ci = partition[ni]
        cj = partition[nj]

        if ci != cj:
            try:
                edges[(ci, cj)] += [(ni, nj)]
            except KeyError:
                edges[(ci, cj)] = [(ni, nj)]

    return edges

def _position_nodes(g, partition, **kwargs):
    """
    Positions nodes within communities.
    """

    communities = dict()
    for node, community in partition.items():
        try:
            communities[community] += [node]
        except KeyError:
            communities[community] = [node]

    pos = dict()
    for ci, nodes in communities.items():
        subgraph = g.subgraph(nodes)
        pos_subgraph = nx.spring_layout(subgraph, **kwargs)
        pos.update(pos_subgraph)

    return pos

def test():
    #to install networkx 2.0 compatible version of python-louvain use:
     
    from community import community_louvain


    partition = community_louvain.best_partition(graph)
    pos = community_layout(graph, partition)

    nx.draw(graph, pos, node_color=list(partition.values())); plt.show()
    return



In [56]:
communities

[{'Bastos de Avila, F.',
  'Bouman, P.J.',
  'Brackel, P.O.M.',
  'Brink, van den , T.',
  'Brink, van den, T',
  'Camillo de Oliveira, A.',
  'Citroen, H.A.',
  'Davis, K.',
  'Edding, F.',
  'Gadolin, de, A.',
  'Glass, D.V.',
  'Groenman, Sj.',
  'HWvanLoon Fellowship, NL regering',
  'Hofstee, E.W.',
  'Institute of International Education',
  'Ipsen, G.',
  'Isaac, J.',
  'Klompe, M.',
  'Lannes, X.',
  'Mill, van, A.N.',
  'Oudegeest, J.J.',
  'Petersen, W.',
  'REMP',
  'Rappard, W.E.',
  'Salin, E.',
  'Sauvy, A.',
  'Steigenga, W.',
  'Verkade, W.',
  'Verwey-Jonker, H.',
  'Wander, H.',
  'Zeegers, G.H.L.'},
 {'Appleyard, R.T.',
  'Balhana, A.Pilatti',
  'Beijer, G.',
  'Beld, van den, C.A.',
  'Belshaw, H.',
  'Borrie, W.D.',
  'Cidor, H.A.',
  'Dorpema, H.',
  'Fagley, R.',
  'Geyl, W.F.',
  'Glinstra Bleeker, van, R.J.P.',
  'Gottmann, J.',
  'Hack, H.',
  'Herberle, R.',
  'Heyn, H.',
  'Hyrenius, H.',
  'Kraak, J.H.',
  'Luckham, B.',
  'Lyon, A.V.',
  'MacKay, M.C.J.',


In [57]:
fig=plt.figure(figsize=(12,8), dpi= 100, facecolor='w', edgecolor='k')

nx.draw_networkx_nodes(graph, pos)
nx.draw_spring(graph, with_labels=False, font_weight='bold')


In [58]:
from scripts.network_analysis import generate_graph, add_entities, add_record_links



periods = [
    {'start': 1950, 'end': 1959}, 
     {'start': 1960, 'end': 1969},
     {'start': 1970, 'end': 1979},
     {'start': 1980, 'end': 1989},
]

grphdict = {}

for period in periods:
    pn = period['start']
    periodgraph = generate_graph()
    for record in relationship_records:
        record['year'] = int(record['year'])
    for ri, record in enumerate(sorted(relationship_records, key = lambda x: x['year'])):
        record['year'] = int(record['year'])
        if record['year'] < period['start'] or record['year'] > period['end']:
            continue
        print(record['year'])
        entities = extract_record_entities(record)
        for entity in entities:
            if entity['entity_type'] == 'person':
                entity['entity_type'] = get_entity_category(entity)
        named_entities = [entity for entity in entities if 'entity_name' in entity]
        print([entity['entity_name'] for entity in named_entities])
        add_entities(periodgraph, named_entities)
        add_record_links(periodgraph, named_entities)
        if ri == 100:
            break
    grphdict[pn] = periodgraph


1951
['REMP', 'Wander, H.', 'Bouman, P.J.', 'Beijer, G.']
adding link REMP Wander, H.
adding link REMP Bouman, P.J.
adding link REMP Beijer, G.
adding link Wander, H. Bouman, P.J.
adding link Wander, H. Beijer, G.
adding link Bouman, P.J. Beijer, G.
1951
['REMP', 'Citroen, H.A.', 'Groenman, Sj.', 'Rappard, W.E.', 'Beijer, G.']
adding link REMP Citroen, H.A.
adding link REMP Groenman, Sj.
adding link REMP Rappard, W.E.
adding link REMP Beijer, G.
adding link Citroen, H.A. Groenman, Sj.
adding link Citroen, H.A. Rappard, W.E.
adding link Citroen, H.A. Beijer, G.
adding link Groenman, Sj. Rappard, W.E.
adding link Groenman, Sj. Beijer, G.
adding link Rappard, W.E. Beijer, G.
1951
['REMP', 'Edding, F.', 'Salin, E.', 'Beijer, G.']
adding link REMP Edding, F.
adding link REMP Salin, E.
adding link REMP Beijer, G.
adding link Edding, F. Salin, E.
adding link Edding, F. Beijer, G.
adding link Salin, E. Beijer, G.
1952
['REMP', 'Beijer, G.', 'Oudegeest, J.J.', 'Sauvy, A.', 'Beijer, G.']
adding 

In [59]:
periodcentralities = {}
nodelists = {}
for pn in grphdict:
    periodcentralities[pn] = nx.eigenvector_centrality(grphdict[pn])
    nodelists[pn] = [n for n in grphdict[pn].nodes()]

In [60]:
from itertools import product, combinations, islice
commonnames = {}

#list(combinations(list(nodelists.keys()), r=3))
# list(islice(list(nodelists.keys()), 3))
window_size = 2

seq = list(nodelists.keys())
for i in range(len(seq) - window_size + 1):
    w = seq[i: i + window_size]
#     {e:set(nodelists[e]) for e in }
    for item in w:
        if item not in commonnames.keys():
            commonnames[item] = set(nodelists[w[0]]) & set(nodelists[w[1]])

In [61]:
colors = {'ICEM':'blue', 'REMP':'orange', 'rest':'black'}

In [84]:
def period_graph(pn, grphdict=grphdict, periodcentralities=periodcentralities, commonnames=commonnames, colors=colors):
    periodgraph = grphdict[pn]

    #first reset label on graph nodes
    for f in periodgraph.nodes():
        periodgraph.nodes[f].update({"community" : ''})

    for f in periodgraph.nodes():
        periodgraph.nodes[f].update({"label":'', 
                                    "centrality" : periodcentralities[pn][f]*4,})
        for i, c in my_communities.items():
            if f in c:
                comty = periodgraph.nodes[f].get('community') or 'rest'
                #comty = '/'.join([comty,i])[1:]
                if f in commonnames[pn]:
                    periodgraph.nodes[f].update({"label":f})
                # else:
                #     label = ''
                # if comty == "rest":
                #     label = ''
                periodgraph.nodes[f].update({"community" : comty,
                                    "edgecolor": colors[i],
                                    "centrality" : periodcentralities[pn][f]*4,
                                    "name" : f,
                                    # "label" : label
                                   })


    partition = community_louvain.best_partition(periodgraph)
    pos = community_layout(periodgraph, partition)
    
    return periodgraph, pos
    

In [85]:
# this does not work
for pn in grphdict:
    periodgraph, pos = period_graph(pn)

    chart = nxa.draw_networkx(
            G=periodgraph,
            pos=pos,
            node_size='centrality',
            node_color='entity_type',
            edge_color='edgecolor',
            cmap='accent',
            node_tooltip=['name'],
            node_label='label',
            font_color="red",
            font_size=12
        ).configure_view(width=400, height=200, )

In [86]:
pn = 1950
periodgraph, pos = period_graph(pn)

chart = nxa.draw_networkx(
            G=periodgraph,
            pos=pos,
            node_size='centrality',
            node_color='entity_type',
            edge_color='edgecolor',
            cmap='accent',
            node_tooltip=['name'],
            node_label='label',
            font_color="red",
            font_size=12
        )
chart.configure_view(width=600, height=400, )


alt.LayerChart(...)

In [88]:
pn = 1960
periodgraph, pos = period_graph(pn)

chart = nxa.draw_networkx(
            G=periodgraph,
            pos=pos,
            node_size='centrality',
            node_color='entity_type',
            edge_color='edgecolor',
            cmap='accent',
            node_tooltip=['name'],
            node_label='label',
            font_color="red",
            font_size=12
        )
chart.configure_view(width=600, height=400, )


alt.LayerChart(...)

In [89]:
pn = 1970
periodgraph, pos = period_graph(pn)

chart = nxa.draw_networkx(
            G=periodgraph,
            pos=pos,
            node_size='centrality',
            node_color='entity_type',
            edge_color='edgecolor',
            cmap='accent',
            node_tooltip=['name'],
            node_label='label',
            font_color="red",
            font_size=12
        )
chart.configure_view(width=600, height=400, )


alt.LayerChart(...)

In [90]:
pn = 1980
periodgraph, pos = period_graph(pn)

chart = nxa.draw_networkx(
            G=periodgraph,
            pos=pos,
            node_size='centrality',
            node_color='entity_type',
            edge_color='edgecolor',
            cmap='accent',
            node_tooltip=['name'],
            node_label='label',
            font_color="red",
            font_size=12
        )
chart.configure_view(width=600, height=400, )


alt.LayerChart(...)